In [29]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [30]:
df = pd.read_csv("dataprep.csv")  

In [31]:
df['Label'] = df.Label.map({'H':0, 'N':1})

In [32]:
df.head()

Label                                          Statement
0      0  @Grumpy_P_Sloth @deanesmay feminists argue for...
1      0  1948Army of England helped the Jews to occupy ...
2      0  Dutch Leader Says Europe to collapse In 6 Week...
3      0  RT @__DeLay: The next day the Romans and the J...
4      0  RT @Bakersman_Joe: When Hitler Invited The Jew...

In [33]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Statement'].values)
X = tokenizer.texts_to_sequences(df['Statement'].values)
X = pad_sequences(X)

In [34]:
print(X)

[[   0    0    0 ... 1192   60  139]
 [   0    0    0 ...    6 1063   38]
 [   0    0    0 ...    9    3    4]
 ...
 [   0    0    0 ...   14    3    4]
 [   0    0    0 ...   14    3    4]
 [   0    0    0 ...  901  124 1738]]


In [35]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 39, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 39, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
Y = pd.get_dummies(data['Label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

NameError: name 'data' is not defined

In [ ]:
print(X_train)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X_train)
rescaledX = scaler.fit_transform(X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X_train)
rescaledX = scaler.transform(X_test)

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

In [ ]:
score,acc=model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print("Score = %.2f" % (score))
print("Accuracy = %.2f" % (acc))